In [5]:
%%time
'''
Importing transaction history data from a cryptocurrency exchange using binance.api
'''
import pandas as pd
import numpy as np
from datetime import datetime
import time
import requests


def convert_time(df, col):
    df[col] = df[col].apply(lambda x: datetime.fromtimestamp(float(x) / 1000))
    return df[col].apply(lambda x: x.strftime('%H:%M:%S')), df[col].apply(lambda x: x.strftime('%Y-%m-%d'))


# Setting deadlines(Jan 1 2020 and Mar 1 2020) for importing data:
curr_time = int(time.mktime((2020, 1, 1, 0, 0, 0, 2, 1, -1))) * 1000
mar_1 = int(time.mktime((2020, 3, 1, 0, 0, 0, 6, 61, -1))) * 1000
params = {'symbol': 'BNBBTC', 'interval': '1m'}

# Creation of data_array which will store the results of the query loop:
data_array = []
while int(curr_time) < mar_1:
    # Changing the parameter and making get-request for transaction history:
    params['startTime'] = curr_time
    response = requests.get('https://api.binance.com/api/v3/klines', params)
    first_500 = response.json()
    data_array += first_500
    curr_time = first_500[-1][0]

# Converting array to dataframe and changing the date style:
df = pd.DataFrame(np.array(data_array)[:, :7],
                  columns=['open_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'close_time'])
df = df.drop_duplicates(subset='open_time')
df['open_time'], df['open_date'] = convert_time(df, 'open_time')
df['close_time'], df['close_date'] = convert_time(df, 'close_time')
df = df[['open_date', 'open_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'close_date',
         'close_time']]
df.index = np.arange(df.shape[0])

# Displaying dataframe and saving to a csv file:
display(df)
df.to_csv('binance.csv')

,open_date,open_time,open_price,high_price,low_price,close_price,volume,close_date,close_time
0,2020-01-01,00:00:00,0.00190040,0.00190060,0.00190040,0.00190060,518.35000000,2020-01-01,00:00:59
1,2020-01-01,00:01:00,0.00190050,0.00190060,0.00190050,0.00190050,424.03000000,2020-01-01,00:01:59
2,2020-01-01,00:02:00,0.00190050,0.00190060,0.00190040,0.00190050,486.57000000,2020-01-01,00:02:59
3,2020-01-01,00:03:00,0.00190040,0.00190060,0.00190040,0.00190040,347.52000000,2020-01-01,00:03:59
4,2020-01-01,00:04:00,0.00190040,0.00190050,0.00190040,0.00190050,189.06000000,2020-01-01,00:04:59
...,...,...,...,...,...,...,...,...,...
86323,2020-03-01,05:37:00,0.00228370,0.00229040,0.00228360,0.00228970,621.53000000,2020-03-01,05:37:59
86324,2020-03-01,05:38:00,0.00229010,0.00229050,0.00228700,0.00229050,859.74000000,2020-03-01,05:38:59
86325,2020-03-01,05:39:00,0.00228940,0.00229210,0.00228490,0.00228490,1480.32000000,2020-03-01,05:39:59
86326,2020-03-01,05:40:00,0.00228450,0.00228840,0.00228400,0.00228840,906.27000000,2020-03-01,05:40:59


Wall time: 1min 30s
